In [2]:
import subprocess

# 解凍されたai.ja.txtファイルのパス
input_file = 'ai.ja.txt'  # 解凍したテキストファイルのパス
output_file = 'ai.ja.txt.parsed'  # 結果を保存するファイル名

# CaboChaを使った解析
# # CaboChaをコマンドラインで実行
cabocha_command = ['cabocha', '-f1', input_file]
with open(output_file, 'w', encoding='utf-8') as outfile:
    subprocess.run(cabocha_command, stdout=outfile)

In [3]:
#40

class Morph:#形態素
    def __init__(self,line):
        part = line.split("\t")
        part_=part[1].split(",")
        self.surface=part[0]
        self.base=part_[-3]
        self.pos=part_[0]
        self.pos1=part_[1]
    
def read_hito_(file_path):
    sentences=[]
    sentence=[]
    with open(file_path,"r",encoding='utf-8') as f:
        for line in f:
            if line[0]=="*":
                continue
            elif line !="EOS\n":
                sentence.append(Morph(line))
            if line =="EOS\n":
                if sentence:
                    sentences.append(sentence)
                    sentence=[]
    return sentences  

pased_file="ai.ja.txt.parsed"
sentences=read_hito_(pased_file)
for i in sentences[2]:
    print(vars(i))

{'surface': '『', 'base': '『', 'pos': '記号', 'pos1': '括弧開'}
{'surface': '日本', 'base': '日本', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '大', 'base': '大', 'pos': '接頭詞', 'pos1': '名詞接続'}
{'surface': '百科全書', 'base': '百科全書', 'pos': '名詞', 'pos1': '一般'}
{'surface': '(', 'base': '*', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'ニッポニカ', 'base': '*', 'pos': '名詞', 'pos1': '一般'}
{'surface': ')』', 'base': '*', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'の', 'base': 'の', 'pos': '助詞', 'pos1': '連体化'}
{'surface': '解説', 'base': '解説', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '情報', 'base': '情報', 'pos': '名詞', 'pos1': '一般'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 'pos1': '一般'}
{'surface': '者', 'base': '者', 'pos': '名詞', 'pos1': '接尾'}
{'surface': '・', 'base': '・', 'pos': '記号', 'pos1': '一般'}
{'surface': '通信', 'base': '通信', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 'pos1'

In [ ]:
#41
   
class Chunk:#文節
    def __init__(self,morphs,dst):
        self.morphs=morphs #（Morphオブジェクト）のリスト
        self.dst=dst #係り先文節インデックス番号
        self.srcs=[] #係り元文節インデックス番号のリスト
        
    def get_surface(self):
        surface=[]
        for morph in self.morphs:
            if morph.pos!="記号":
                surface.append(morph.surface)
        result="".join(surface)
        return result
    
class Sentence:#文
    def __init__(self,chunks):
        self.chunks=chunks
        for i, chunk in enumerate(self.chunks):
            if chunk.dst!=-1:  # 有効な係り先文節の場合
                self.chunks[chunk.dst].srcs.append(i)  # 係り先文節のsrcsに係り元の文節インデックスiを追加

        
def read_hito(file_path):
    sentences=[]#文
    chunks=[]#文節
    morphs=[]#形態素
    dst=-1#インデックス番号
    
    with open(file_path,"r",encoding='utf-8') as f:
        for line in f:
            if line[0]=="*":
                if morphs:
                    chunks.append(Chunk(morphs,dst))
                    morphs=[]
                dst = int(line.split(' ')[2].replace("D", ""))
            elif line !="EOS\n":
                morphs.append(Morph(line))
            if line =="EOS\n":
                if morphs:
                    chunks.append(Chunk(morphs,dst))
                if chunks:
                    sentences.append(Sentence(chunks))
                chunks=[]
                morphs=[]
                dst=-1
    return sentences  

pased_file="ai.ja.txt.parsed"
sentences=read_hito(pased_file)
for i, chunk in enumerate(sentences[2].chunks):
    surface = chunk.get_surface()
    dst = chunk.dst
    if dst != -1 and dst < len(sentences[2].chunks):  # 範囲外を防いでいるか
        dst_surface = sentences[2].chunks[dst].get_surface()
    else:
        dst_surface = "なし"
    print(f"文節 {i}: '{surface}' -> 係り先: '{dst_surface}'")

文節 0: '日本大百科全書(ニッポニカ)』の' -> 係り先: '解説で'
文節 1: '解説で' -> 係り先: '述べている'
文節 2: '情報工学者通信工学者の' -> 係り先: '佐藤理史は'
文節 3: '佐藤理史は' -> 係り先: '述べている'
文節 4: '次のように' -> 係り先: '述べている'
文節 5: '述べている' -> 係り先: 'なし'


In [8]:
#42
for chunk in sentences[2].chunks:
    src_surface = chunk.get_surface()
    if chunk.dst != -1:  # 係り先がある場合
        dst_surface = sentences[2].chunks[chunk.dst].get_surface()
        print(f"{src_surface}\t{dst_surface}")

日本大百科全書(ニッポニカ)』の	解説で
解説で	述べている
情報工学者通信工学者の	佐藤理史は
佐藤理史は	述べている
次のように	述べている


In [11]:
#43

for chunk in sentences[2].chunks:
    src_surface = chunk.get_surface()
    if chunk.dst != -1:  # 係り先がある場合
        dst_surface = sentences[2].chunks[chunk.dst].get_surface()
        for morph in chunk.morphs:
            if morph.pos=="名詞":
                for morph in sentences[2].chunks[chunk.dst].morphs:
                    if morph.pos=="動詞":
                        print(f"{src_surface}\t{dst_surface}")


解説で	述べている
解説で	述べている
佐藤理史は	述べている
佐藤理史は	述べている
佐藤理史は	述べている
佐藤理史は	述べている
佐藤理史は	述べている
佐藤理史は	述べている
次のように	述べている
次のように	述べている
次のように	述べている
次のように	述べている


In [6]:
%pip install graphviz

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
from graphviz import Digraph

# 簡単なグラフを作成
dot = Digraph()
dot.node('A', 'Hello')
dot.node('B', 'World')
dot.edge('A', 'B')

# グラフをファイルとして保存
dot.render('test_graph', format='png', view=False)
print("グラフが生成されました。")



グラフが生成されました。


In [ ]:
#44
from graphviz import Digraph

def visualize_dependency_tree(sentence, output_file="dependency_tree"):
    # 有向グラフを作成
    dot = Digraph(format="png")
    dot.attr('node', shape='box', fontname="MS Gothic")  # フォントを日本語対応に設定
    
    # 文節をノードとして追加し、係り受け関係をエッジで追加
    for i, chunk in enumerate(sentence.chunks):
        surface = chunk.get_surface()
        dot.node(f"{i}", surface)  # ノードのラベルは文節の表層形
        if chunk.dst != -1:  # 有効係り先がある場合
            dst_surface = sentence.chunks[chunk.dst].get_surface()
            dot.edge(f"{i}", f"{chunk.dst}")  # エッジを追加

    dot.render(output_file, cleanup=True)
    print(f"係り受け木を'{output_file}.png'として保存しました。")

visualize_dependency_tree(sentences[2], output_file="sentence2_dependency_tree")



係り受け木を'sentence2_dependency_tree.png'として保存しました。


In [ ]:
#45
from collections import defaultdict

# 出力をファイルに保存しつつ、printで表示
with open('./ans45.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        # 文の中の各chunkを処理
        for chunk in sentence.chunks:
            # 文節内の最左の動詞を取得
            verb = None
            for morph in chunk.morphs:
                if morph.pos == '動詞':  # 動詞を探す
                    verb = morph.base  # 動詞の基本形を取得
                    break  # 最左の動詞を見つけたら処理を終了
            
            if verb==None:  # 動詞がない場合はスキップ
                continue

            cases = []  # 助詞リスト
            for src in chunk.srcs:
                # 範囲内の文節を確認して処理
                if 0 <= src < len(sentence.chunks):
                    src_chunk = sentence.chunks[src]  # 係り元の文節
                    # 助詞を抽出
                    particles = [m.surface for m in src_chunk.morphs if m.pos == '助詞']
                    if particles:  # 助詞が見つかった場合
                        cases.extend(particles)  # 助詞をリストに追加

            # 助詞が見つかった場合のみ出力
            if cases:
                # 重複を除去し辞書順にソート
                cases = sorted(set(cases))
                # 出力形式: 述語（動詞） + 助詞
                line = f"{verb}\t{' '.join(cases)}"

                # ファイルに書き込む
                print(line, file=f)

                if sentence==sentences[2]:
                    print(line)



用いる	を
する	て を
指す	を
代わる	に を
行う	て に
する	と も


In [16]:
!cat "/c/Users/横山　明咲/100knock2024_huyu/chapter05/ans45.txt" | grep '^行う' | sort | uniq -c | sort -nr   
!cat "/c/Users/横山　明咲/100knock2024_huyu/chapter05/ans45.txt" | grep '^なる' | sort | uniq -c | sort -nr
!cat "/c/Users/横山　明咲/100knock2024_huyu/chapter05/ans45.txt" | grep '^与える' | sort | uniq -c | sort -nr      

#文字化けが発生したため、gitbushで実行したら出力された
#出力例（与える））
#      1 与える  に は を
#      1 与える  が に
#      1 与える  が など に

'cat' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'cat' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'cat' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [ ]:
#46
# 出力をファイルに保存しつつ、printで表示
with open('./ans46.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        # 文の中の各chunkを処理
        for chunk in sentence.chunks:
            # 文節内の最左の動詞を取得
            verb = None
            for morph in chunk.morphs:
                if morph.pos == '動詞':  # 動詞を探す
                    verb = morph.base  # 動詞の基本形を取得
                    break  # 最左の動詞を見つけたら処理を終了
            
            if verb==None:  # 動詞がない場合はスキップ
                continue

            # 動詞に係る文節の助詞と項を収集
            cases_and_items = []  # 格（助詞）と項（文節そのもの）のペア
            for src in chunk.srcs:
                # 範囲内の文節を確認して処理
                if 0 <= src < len(sentence.chunks):
                    src_chunk = sentence.chunks[src]  # 係り元の文節
                    # 助詞を抽出
                    particles = [m.surface for m in src_chunk.morphs if m.pos == '助詞']
                    if particles:  # 助詞が見つかった場合
                        item = ''.join(m.surface for m in src_chunk.morphs)  # 文節そのもの（単語列）
                        cases_and_items.append((particles[-1], item))  # 最右の助詞を格とする

            # 格と項を辞書順に整列
            if cases_and_items:
                # 助詞を基準にソート
                cases_and_items = sorted(cases_and_items, key=lambda x: x[0])
                cases = [ci[0] for ci in cases_and_items]  # 助詞
                items = [ci[1] for ci in cases_and_items]  # 文節

                # 出力形式: 述語（動詞） + 助詞 + 項
                line = f"{verb}\t{' '.join(cases)}\t{' '.join(items)}"

                # ファイルに書き込む
                print(line, file=f)

                # Notebookに出力として表示
                print(line)



述べる	で に は	解説で、 次のように 佐藤理史は


In [20]:
#47
from collections import defaultdict

with open('./ans47.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        for chunk in sentence.chunks:
            # 「サ変接続名詞+を」で構成される文節か判定
            sahen_wo = None
            for i, morph in enumerate(chunk.morphs[:-1]):  # 最後の形態素を含めない
                if morph.pos1 == 'サ変接続' and chunk.morphs[i + 1].surface == 'を':
                    sahen_wo = morph.surface + chunk.morphs[i + 1].surface
                    break

            if not sahen_wo:  # 該当しない場合は次の文節へ
                continue

            # 「サ変接続名詞+を」の係り先に動詞があるかを判定
            verb = None
            dst_chunk = sentence.chunks[chunk.dst] if chunk.dst != -1 else None
            if dst_chunk:  # 係り先が存在する場合
                for morph in dst_chunk.morphs:
                    if morph.pos == '動詞':  # 動詞を探す
                        verb = morph.base  # 動詞の基本形
                        break

            if not verb:  # 動詞がない場合は次の文節へ
                continue

            # 述語を生成
            predicate = f"{sahen_wo}{verb}"

            # 助詞と項の収集
            cases_and_items = []
            for src in dst_chunk.srcs:
                if src == chunk.dst or src == sentence.chunks.index(chunk):  # 自身の文節は無視
                    continue
                src_chunk = sentence.chunks[src]
                particles = [m.surface for m in src_chunk.morphs if m.pos == '助詞']
                if particles:
                    item = ''.join(m.surface for m in src_chunk.morphs)
                    cases_and_items.append((particles[-1], item))

            if cases_and_items:
                # 助詞と項を辞書順にソート
                cases_and_items.sort(key=lambda x: x[0])
                cases = [ci[0] for ci in cases_and_items]
                items = [ci[1] for ci in cases_and_items]

                # 出力形式: 述語 + 助詞 + 項
                line = f"{predicate}\t{' '.join(cases)}\t{' '.join(items)}"
                print(line, file=f)
                print(line)  # 確認用



行動を代わる	に	人間に
記述をする	と	主体と
注目を集める	が	「サポートベクターマシン」が
経験を行う	に を	元に 学習を
学習を行う	に を	元に 経験を
学習をする	て に は を を通して	なされている。 元に ACT-Rでは、 推論ルールを、 生成規則を通して
進化を見せる	て て において は	活躍している。 加えて、 生成技術において （敵対的生成ネットワーク）は、
開発を行う	は	エイダ・ラブレスは
処理を行う	に に により	同年に、 Webに ティム・バーナーズ＝リーにより、
意味をする	に	データに
処理を行う	て に	付加して、 コンピュータに
研究を進める	て	費やして
命令をする	で	機構で
運転をする	に	元に
特許をする	が に	日本が 2018年までに
運転をする	て に	基づいて、 柔軟に
注目を集める	から は	ことから、 ファジィは
制御を用いる	て も	受けて、 他社も
制御をする	から	少なさから、
改善を果たす	が で に	チームが 画像処理コンテストで 2012年に
研究を続ける	が て	ジェフ・ホーキンスが、 向けて
)をする	は	8月には、
注目を集める	に	急速に
投資を行う	で に	民間企業主導で 全世界的に
探索を行う	で	無報酬で
推論をする	て	経て
研究を始める	とも は	マックスプランク研究所とも Googleは、
研究を行う	て	始めており、
開発をする	で は	官民一体で 中国では
開発をする	で	日本で
投資をする	に は	2022年までに 韓国は、
反乱を起こす	て に対して	於いて、 人間に対して
監視を行う	に まで	人工知能に 歩行者まで
手続きを経る	を	ウイグル族を
制御をする	は	AIプログラムは
判断を介す	から	観点から
禁止を求める	が は	ヒューマン・ライツ・ウォッチが、 4月には
競争を行う	は をめぐって	米国・中国・ロシアは 軍事利用をめぐって
追及を受ける	て で と とともに は	暴露されており、 整合性で 拒否すると 「」とともに、 公聴会では、
研究をする	が	Microsoftが
解任をする	て は	含まれており、 Google社員らは
解散をする	が で は	倫理委員会が 理由で、 Googleは
存在を見いだす	に	ものに
話をする	は	哲学者は


In [25]:
#48
sentence=sentences[2]    
for chunk in sentence.chunks:
        # 文節に名詞を含む場合のみ処理
        if any(morph.pos == "名詞" for morph in chunk.morphs):
                path = []  # パスを格納するリスト
                current_chunk = chunk

                # 構文木の根に至るまでパスをたどる
                while current_chunk:
                    # 文節の表層形を取得し、記号を除外
                    surface = ''.join(morph.surface for morph in current_chunk.morphs if morph.pos != '記号')

                    # パスに追加
                    path.append(surface)

                    # 次の文節へ（係り先が -1 の場合は終了）
                    if current_chunk.dst == -1:
                        break
                    current_chunk = sentence.chunks[current_chunk.dst]

                # パスを " -> " で連結
                path_str = " -> ".join(path)

                # 結果をコンソールに出力
                print(path_str)

日本大百科全書(ニッポニカ)』の -> 解説で -> 述べている
解説で -> 述べている
情報工学者通信工学者の -> 佐藤理史は -> 述べている
佐藤理史は -> 述べている
次のように -> 述べている


In [21]:
#49
from itertools import combinations
import re

sentence = sentences[2]
nouns = []

# 名詞を含む文節のインデックスを取得
for i, chunk in enumerate(sentence.chunks):
    if any(morph.pos == "名詞" for morph in chunk.morphs):  # 名詞を含む文節を抽出
        nouns.append(i)

# 名詞を含む文節ペアごとにパスを作成
for i, j in combinations(nouns, 2):
    path_I = []
    path_J = []

    # 文節iと文節jがどのように構文木上で繋がるかを確認
    while i != j:
        if i < j:  # 文節iの構文木経路上に文節jが存在する場合
            path_I.append(i)
            i = sentence.chunks[i].dst
        else:  # 文節iの構文木経路上に文節jがない場合
            path_J.append(j)
            j = sentence.chunks[j].dst

    # 文節iの構文木上に文節jが存在する場合
    if len(path_J) == 0:
        X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"]) 
        Y = "Y" + "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "名詞" and morph.pos != "記号"])
        chunk_X = re.sub("X+", "X", X)
        chunk_Y = re.sub("Y+", "Y", Y)
        
        # 文節iから文節jに至る経路を表示
        path_ItoJ = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs if morph.pos != "記号")] + [chunk_Y]
        print(" -> ".join(path_ItoJ))
    
    # 文節iの構文木上に文節jが存在しない場合
    else:
        X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"]) 
        Y = "Y" + "".join([morph.surface for morph in sentence.chunks[path_J[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"]) 
        chunk_X = re.sub("X+", "X", X)
        chunk_Y = re.sub("Y+", "Y", Y)
        chunk_k = "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "記号"])

        # 共通の文節で分けて表示
        path_X = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
        path_Y = [chunk_Y] + ["".join(morph.surface for n in path_J[1:] for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
        
        # 結果を表示
        print(" | ".join([" -> ".join(path_X), " -> ".join(path_Y), chunk_k]))





X大の ->  -> Yで
X大の -> 解説で | Yの -> 佐藤理史は | 述べている
X大の -> 解説で | Yは ->  | 述べている
X大の -> 解説で | Yのに ->  | 述べている
Xで ->  | Yの -> 佐藤理史は | 述べている
Xで ->  | Yは ->  | 述べている
Xで ->  | Yのに ->  | 述べている
Xの ->  -> Yは
Xの -> 佐藤理史は | Yのに ->  | 述べている
Xは ->  | Yのに ->  | 述べている
